# Agents d'Intelligence Artificielle

**Introduction**  

Les agents d'intelligence artificielle (IA) sont des systèmes autonomes conçus pour effectuer des tâches spécifiques sans intervention humaine constante. Leur importance s'est accrue dans divers domaines, notamment le service client, l'automatisation des tâches, et les assistants personnalisés.
Dans cet atelier, nous allons explorer comment créer des agents IA spécialisés, intégrés avec des modèles LLM, pour des performances optimales dans des contextes spécifiques tels que l'assistance au codage ou l'éducation.

- Comparison of AI Agent Frameworks

| Framework	| Key Focus |
| ---       | ---       |
| CrewAI	| Role-playing AI agents |
| LangGraph	| Stateful multi-actor systems |
| Microsoft Semantic Kernel	| Enterprise AI integration |
| Microsoft Autogen	| Multi-agent conversational systems | 

**Principles of AI Agents** 

- Core Characteristics:
Autonomy, adaptability, interactivity, and goal-orientation.

- Types of AI Agents:
Reactive agents.
Cognitive agents.
Hybrid agents.

- Behavior Modeling:
Rule-based systems vs. learning-based systems.

**Architectures of AI Agents**

- Fundamental Architectures:
Reactive architecture.
Deliberative architecture.
Hybrid architecture.

- Agent Communication:
Agent-to-agent protocols.
Natural language processing for interaction.

- Scalability and Integration:
Multi-agent systems (MAS).
Coordination and cooperation mechanisms.

**Pré-requis**




In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import requests 
from langchain_community.llms.ollama import Ollama
from crewai import Agent, Task, Crew, Process
import os
from dotenv import load_dotenv


## Ollama pour les exécutions **locales**
Ollama offre une méthode efficace et simple pour exécuter des modèles de langage localement sur votre machine, sans dépendre d'une infrastructure cloud. En hébergeant le modèle localement, Ollama permet de traiter les requêtes avec une faible latence, de garantir la confidentialité des données et de travailler hors ligne. L'installation typique consiste à exécuter le serveur Ollama sur votre machine locale et à interagir avec lui via une API, par exemple en envoyant des requêtes HTTP au serveur local à http://localhost:<port>/api/generate. Cette approche est idéale pour les environnements où le contrôle des ressources de calcul et la sécurité des données sont prioritaires.

⚠️ **Si vous utilisez Ollama en local, n’oubliez pas de décommenter les lignes correspondantes et de commenter celles liées au cloud.**

In [13]:
# Connecter le modèle Ollama
llm =  Ollama(model="llama3.2:1b")

In [6]:
llm.invoke("Hello, how are you?")

"I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm here and ready to help you with any questions or topics you'd like to discuss. How about you? How's your day going so far?"

> Build a simple goal-oriented chatbot

In [13]:
def chatbot_conversation():
    conversation_history = []
    
    # Start the conversation
    while True:
        user_input = input("You: ")
        conversation_history.append(f"You: {user_input}")
        print(f"You: {user_input}")
        
        # Combine previous conversation history into a single prompt for better context
        prompt = "\n".join(conversation_history) + "\nAI:"
        
        response = llm(prompt)
        conversation_history.append(f"AI: {response}")
        
        print(f"AI: {response}")
        
        if user_input.lower() == "exit":
            break

# Uncomment the line below to start the conversation
chatbot_conversation()


You: hi
AI: Hi. Is there something I can help you with or would you like to chat?
You: what is NLP
AI: Hi again! 

NLP stands for Natural Language Processing, which is a subfield of artificial intelligence (AI) that deals with the interaction between computers and human language. It's a crucial aspect of machine learning, as it enables computers to understand, interpret, and generate human language.

Think of NLP like a conversation assistant. You can ask me questions, provide input, or even tell me a story, and I'll do my best to respond in a way that makes sense. My goal is to comprehend the context and meaning behind your words, so I can provide accurate and relevant information.

Some common tasks associated with NLP include:

* Sentiment analysis: determining the emotional tone of a piece of text
* Text classification: categorizing text into predefined categories (e.g., spam vs. not spam)
* Translation: translating text from one language to another
* Summarization: summarizing lon

## ChatGPT est-il un agent d'intelligence artificielle ?  

Bien qu'il présente certaines caractéristiques d'un agent d'IA, comme l'exécution de tâches, la génération de texte, l'aide à la résolution de problèmes, l'analyse de données et la réponse à des questions, ChatGPT ne peut pas automatiser les tâches. ChatGPT est un chatbot d'IA qui ne fonctionne qu'avec des commandes et ne peut pas s'intégrer à ton entreprise, ce qui fait qu'il est loin d'être un agent d'IA. Si tu cherches un agent d'IA qui peut s'intégrer à ton entreprise et automatiser des tâches telles que la rédaction de courriels, l'analyse de données, la documentation et la gestion des connaissances, TextCortex est la voie à suivre.



`Core Characteristics`:  
- Autonomy, adaptability, interactivity, and goal-orientation.

In [1]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool(file_path='data/AI_Stories.txt')

In [11]:
# Agents
researcher = Agent(
  role="{topic} Senior Data Researcher",
  goal="Uncover developments in {topic} using the file_read_tool provided in this knowledge base ",
  tools = [file_read_tool],
  verbose=True,
  backstory="You're a seasoned researcher with a knack for uncovering the latest"
    "developments in {topic}. Known for your ability to find the most relevant"
    "information and present it in a clear and concise manner.",
  llm= llm
)

reporting_analyst = Agent(
  role= "{topic} Reporting Analyst",
  goal= "Create detailed reports based on {topic} data analysis and research findings",
  verbose=True,
  backstory= "You're a meticulous analyst with a keen eye for detail. You're known for"
    "your ability to turn complex data into clear and concise reports, making"
    "it easy for others to understand and act on the information you provide.",
   llm= llm
)

In [12]:
# Tasks

research_task = Task(
  description="Conduct a thorough research about {topic}"
    "Make sure you find any interesting and relevant information given"
    "the current year is 2025.",
  expected_output="A list with 10 bullet points of the most relevant information about {topic}",
  agent= researcher
)

reporting_task = Task(
  description="Review the context you got and expand each topic into a full section for a report."
    "Make sure the report is detailed and contains any and all relevant information.",
  expected_output="A fully fledge reports with the mains topics, each with a full section of information."
    "Formatted as markdown without '```'",
  agent= reporting_analyst,
  output_file= "report.md"
)

In [13]:
# Crew
Crew(
      agents= [researcher, reporting_analyst],
      tasks= [research_task, reporting_task],
      verbose=True,
    ).kickoff({"topic": "AI"})

 [DEBUG]: == Working Agent: AI Senior Data Researcher
 [INFO]: == Starting Task: Conduct a thorough research about AIMake sure you find any interesting and relevant information giventhe current year is 2025.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by gathering information on AI developments in 2025.

Initial Research:
 
- According to reports, Google announced its latest AI chip, the Tensor Processing Unit (TPU), which is designed for deep learning tasks.
- Microsoft has developed a new AI framework called Azure Machine Learning for building and deploying machine learning models at scale.
- Amazon has acquired several AI startups in 2025, including an autonomous driving company.

Action: Read a file's content(content.txt) - A tool that can be used to read data/AI_Stories.txt's content

Input:
 Action: Read a file's content
 Action Input: {
 "tool": "content.txt",
 "file_path": "/path/to/ai_stories.txt"
}
 

Action 'Read a file's content(content.txt) - A too

"In 2025, significant advancements were made in AI development. This year saw notable developments across various industries, including the advancement of deep learning technology with Google's Tensor Processing Unit (TPU). The TPU is designed specifically for deep learning tasks and has been a cornerstone in improving machine learning capabilities.\n\nMicrosoft also continued to push the boundaries of AI innovation. Azure Machine Learning emerged as an essential tool for building and deploying complex machine learning models at scale. This development not only aids in efficient data processing but also ensures seamless integration with various systems, making it ideal for industries requiring advanced analytical capabilities.\n\nAmazon played a crucial role in advancing AI research, acquiring several startups specializing in autonomous driving technologies. These acquisitions not only contributed to the growth of the artificial intelligence sector but also paved the way for innovative

`Types of AI Agents`:
- Reactive agents.
- Cognitive agents.
- Hybrid agents.

*Reactive Agents*

- Definition: Reactive agents are the simplest type of AI agents. They respond directly to stimuli from the environment without maintaining any internal state or memory of past actions.
- Characteristics:  
    - Lack memory or history: They don't recall previous states or learn from past experiences.
    - Immediate response: Their behavior is entirely driven by the current input.
    - Simple and fast: Since they don't process complex data or plans, they are computationally efficient.




In [59]:
import random

# Define the grid
grid = [
    ['.', '.', '.', '#', '.'],
    ['.', '#', '.', '#', '.'],
    ['.', '.', 'A', '.', 'G'],
    ['.', '#', '.', '.', '.'],
    ['.', '.', '.', '#', '.']
]

# Display the grid
def display_grid(grid):
    for row in grid:
        print(" ".join(row))
    print("\n")

# Find the agent's position
def find_position(grid, target):
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if cell == target:
                return i, j
    return None

# Move the agent
def move_agent(grid, direction):
    x, y = find_position(grid, 'A')
    grid[x][y] = '.'  # Clear the current position

    if direction == 'UP' and x > 0 and grid[x-1][y] != '#':
        x -= 1
    elif direction == 'DOWN' and x < len(grid)-1 and grid[x+1][y] != '#':
        x += 1
    elif direction == 'LEFT' and y > 0 and grid[x][y-1] != '#':
        y -= 1
    elif direction == 'RIGHT' and y < len(grid[0])-1 and grid[x][y+1] != '#':
        y += 1

    grid[x][y] = 'A'  # Update new position

# Reactive agent's logic
def reactive_agent(grid):
    while True:
        display_grid(grid)
        agent_pos = find_position(grid, 'A')
        goal_pos = find_position(grid, 'G')

        if goal_pos == None:
            print("Goal reached!")
            break

        # Simple reactive behavior: move toward the goal
        x_a, y_a = agent_pos
        x_g, y_g = goal_pos

        if x_g < x_a:  # Move up
            move_agent(grid, 'UP')
        elif x_g > x_a:  # Move down
            move_agent(grid, 'DOWN')
        elif y_g < y_a:  # Move left
            move_agent(grid, 'LEFT')
        elif y_g > y_a:  # Move right
            move_agent(grid, 'RIGHT')

# Run the reactive agent
reactive_agent(grid)


. . . # .
. # . # .
. . A . G
. # . . .
. . . # .


. . . # .
. # . # .
. . . A G
. # . . .
. . . # .


. . . # .
. # . # .
. . . . A
. # . . .
. . . # .


Goal reached!


*Cognitive Agents*  

- Definition: Cognitive agents are more advanced than reactive agents. They can reason, learn, and plan based on past experiences and future goals.
- Characteristics:
    - Memory: They maintain a history of past actions and use it to inform future decisions.
    - Learning: They adapt and improve over time by analyzing outcomes and patterns.
    - Planning: They can anticipate future states and make decisions to achieve long-term goals.

In [8]:
# Tool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
load_dotenv()
search_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()

company_name = "Orange"

# Researcher Agent (Gathers information about the company's reputation)
researcher = Agent(
    role="Reputation Researcher",
    goal="Search for information about {company_name} and gather relevant data on its reputation.",
    verbose=True,
    backstory="You're an experienced researcher skilled in gathering insights from web. You specialize in collecting relevant information about brand reputation.",
    llm=llm,  # Make sure 'llm' is correctly initialized for language processing
    tools=[search_tool, scraper_tool],  # Providing the search tool to the agent
    allow_delegation=False
)

# Task for Researcher Agent (Gathering data on the company's reputation)
research_task = Task(
    description="""Search for {company_name} on various online platforms. 
    Gather all relevant data on the company's online reputation, including sentiment (positive, negative, neutral) for each piece of feedback, 
    major discussions, and any recent reputation-related events.""",
    expected_output="""A list of relevant information, including:
    - Key news articles or mentions
    - Sentiment (positive, negative, neutral) for each piece of feedback
    - Major discussions or events impacting the reputation of the company""",
    agent=researcher
)


# Writer Agent (Generates the report based on the research data)
writer = Agent(
    role="Reputation Report Writer",
    goal="Analyze the research findings and write a detailed report summarizing the online reputation and sentiment surrounding {company_name}.",
    verbose=True,
    backstory="You're a skilled writer who specializes in generating clear and detailed reports from research findings. You excel at summarizing complex data and presenting it in an easily understandable way.",
    llm=llm,  # Make sure 'llm' is correctly initialized for language processing
    allow_delegation=False
)

# Task for Writer Agent (Creating the reputation report)
writer_task = Task(
    description="""Based on the research data provided, write a detailed report that includes:
    - Sentiment breakdown (positive, negative, neutral) for each feedback item
    - Key trends in public opinion
    - Major discussions or viral events that could impact reputation
    - Potential PR issues or crises""",
    expected_output="""A markdown report summarizing the online reputation and sentiment surrounding {company_name}. 
    The report should include:
    - Sentiment breakdown for each piece of feedback (positive, negative, neutral)
    - Key trends and major discussions
    - Any reputation-related events or PR issues""",
    agent=writer,
    context=[research_task],  # Providing the research task as context for the writer
    output_file=f'{company_name}reputation_report.md'  # Save the report to this file
)



In [9]:

Crew(
    agents=[ researcher, writer],
    tasks=[ research_task, writer_task],
    verbose=True
    ).kickoff({"company_name": company_name})

2025-01-14 13:47:35,222 - 28384 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Reputation Researcher
 [INFO]: == Starting Task: Search for Orange on various online platforms. 
    Gather all relevant data on the company's online reputation, including sentiment (positive, negative, neutral) for each piece of feedback, 
    major discussions, and any recent reputation-related events.


> Entering new CrewAgentExecutor chain...
Thought: 

Action: Search the internet
Action Input:
{
    "search_query": "orange"
}
 


Search results: Title: Corporate Website of Orange
Link: https://www.orange.com/en
Snippet: Welcome to the corporate website of the Orange Group, one of the world's leading telecommunications and digital service provider. Orange is here.
---
Title: Orange (fruit) - Wikipedia
Link: https://en.wikipedia.org/wiki/Orange_(fruit)
Snippet: The orange, also called sweet orange to distinguish it from the bitter orange (Citrus × aurantium), is the fruit of a tree in the family Rutaceae.
---
Title: Orange Amps – British Guitar Amps
Link

"my best complete final answer to the task.\n\nThought: After conducting a thorough analysis of online reviews and feedback, I have compiled a detailed report on the current state of Orange's reputation."

*Hybrid Agents*  

- Definition: Hybrid agents combine the best features of reactive and cognitive agents. They can quickly react to immediate stimuli while also reasoning and planning based on memory and goals.

- Characteristics:
    - Layered architecture: Hybrid agents often separate reactive components (for quick decisions) from cognitive components (for strategic thinking).
    - Balance: They can handle both short-term and long-term decision-making.
Complex tasks: Well-suited for scenarios requiring both immediate responses and thoughtful planning.

In [53]:
# Tic Tac Toe Game

# Initialize Tic Tac Toe Board
board = [" " for _ in range(9)]  # 3x3 board represented as a list


def display_board():
    """Displays the current state of the board."""
    print("\n")
    for i in range(3):
        print(f" {board[3 * i]} | {board[3 * i + 1]} | {board[3 * i + 2]} ")
        if i < 2:
            print("---+---+---")
    print("\n")


def is_winner(mark):
    """Checks if the given mark ('X' or 'O') has won."""
    winning_combinations = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
        [0, 4, 8], [2, 4, 6],             # diagonals
    ]
    return any(all(board[pos] == mark for pos in combo) for combo in winning_combinations)


def is_draw():
    """Checks if the game is a draw."""
    return " " not in board

In [54]:
from crewai_tools  import tool

# Tools for the AI Agent
@tool("Place Mark")
def place_mark(position: int) -> str:
    """
    Places the AI's mark ('O') on the given position.
    Input: Integer position (0-8).
    Returns: Success or error message.
    """
    if not isinstance(position, int):
        return f"Invalid input type. Position must be an integer. Received: {type(position)}"
    if 0 <= position < 9:
        if board[position] == " ":
            board[position] = "O"
            return f"Mark placed at position {position}."
        else:
            return f"Position {position} is already occupied."
    return f"Invalid position {position}. Choose a number between 0 and 8."


@tool("Inspect Board")
def inspect_board(_) -> str:
    """Returns the current state of the Tic Tac Toe board."""
    return f"Board state: {board}"

In [55]:
# Define the AI Agent
ai_agent = Agent(
    role="Tic Tac Toe AI",
    goal="Make the best move as 'O' based on the current state of the board. use the 'Inspect Board' tool to evaluate the board state and 'Place Mark(position)' tool to make a move.",
    tools=[inspect_board, place_mark],
    verbose=True,
    backstory="You're an expert in Tic Tac Toe, making strategic decisions to beat the human player.",
    llm=llm
)


# Define the Task for the AI Agent
ai_move_task = Task(
    description=(
        "Evaluate the current board state using the 'Inspect Board' tool. "
        "Determine the best possible move as 'O' based on game rules: prioritize winning moves, "
        "block opponent's winning moves, or pick strategic positions. "
        "return the position where 'O' should be placed on the board."
    ),
    expected_output="The position where 'O' should be placed on the board. make sure it's just a number between 0 and 8. return just the number.",
    agent=ai_agent
)



In [56]:
# Crew setup
crew = Crew(
    agents=[ai_agent],
    tasks=[ai_move_task],
    verbose=True
)

# AI Move Handler in Game Loop
def ai_make_move():
    """Handles the AI's move using the Crew agent."""
    result = crew.kickoff({})
    print(result)  # Print AI's move or errors from the Crew interaction.
    try:
        
        move = int(result)
        if 0 <= move < 9 and board[move] == " ":
            board[move] = "O"
    except ValueError:
        print("AI made an invalid move. Skipping turn.")
    display_board()

# Improved Game Loop
def play_tic_tac_toe():
    print("Welcome to Tic Tac Toe! You are 'X', and the AI is 'O'.")
    display_board()
    player_turn = True

    while True:
        if player_turn:
            # Player's move
            try:
                move = int(input("Enter your move (0-8): "))
                if 0 <= move < 9 and board[move] == " ":
                    board[move] = "X"
                    display_board()
                    if is_winner("X"):
                        print("Congratulations! You won!")
                        break
                    player_turn = False
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Invalid input. Please enter a number between 0 and 8.")
        else:
            # AI's move
            ai_make_move()
            if is_winner("O"):
                print("The AI wins! Better luck next time.")
                break
            player_turn = True

        if is_draw():
            print("It's a draw!")
            break


# Start the game
play_tic_tac_toe()

2025-01-13 21:20:24,064 - 29668 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Welcome to Tic Tac Toe! You are 'X', and the AI is 'O'.


   |   |   
---+---+---
   |   |   
---+---+---
   |   |   




   | X |   
---+---+---
   |   |   
---+---+---
   |   |   


 [DEBUG]: == Working Agent: Tic Tac Toe AI
 [INFO]: == Starting Task: Evaluate the current board state using the 'Inspect Board' tool. Determine the best possible move as 'O' based on game rules: prioritize winning moves, block opponent's winning moves, or pick strategic positions. return the position where 'O' should be placed on the board.


> Entering new CrewAgentExecutor chain...
Thought: I now know the final answer
Final Answer: 5

> Finished chain.
 [DEBUG]: == [Tic Tac Toe AI] Task output: 5


5


   | X |   
---+---+---
   |   | O 
---+---+---
   |   |   




   | X | X 
---+---+---
   |   | O 
---+---+---
   |   |   


 [DEBUG]: == Working Agent: Tic Tac Toe AI
 [INFO]: == Starting Task: Evaluate the current board state using the 'Inspect Board' tool. Determine the best possible move as 'O' base

## Use case : Job Application Crew

> Setup

In [3]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)


> Tools

In [9]:
read_resume = FileReadTool(file_path='data/User_Resume.md')
    

> Agents

In [14]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scraper_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm = llm
)


# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scraper_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = llm
)


# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scraper_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm = llm
)

# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scraper_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = llm
)

> Tasks

In [15]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


> Execution aet Resultats

In [17]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

job_application_inputs = {
    'job_posting_url': 'https://ma.indeed.com/viewjob?jk=eda451f078ae7d00&from=shareddesktop',
    'github_url': 'https://github.com/mohannadtazi',
    'personal_writeup': """As a Master's student in Data Science and Big Data at the Faculty of Sciences Ben M'Sick, Université Hassan II Casablanca, I have developed a deep passion for leveraging advanced technologies to address real-world challenges. My academic journey and hands-on projects have equipped me with a solid foundation in machine learning, artificial intelligence, and data-driven problem-solving.
Throughout my studies, I have led and contributed to various academic and professional initiatives, including the development of a rainfall prediction classifier, which utilized multiple machine learning algorithms and evaluation metrics. This project honed my technical skills in Python, data visualization, and performance evaluation, while emphasizing the importance of building scalable, reliable solutions.
I have also authored a scientific paper on the application of generative AI in education, focusing on personalized learning through specialized large language models (LLMs), Retrieval-Augmented Generation (RAG), and prompt engineering. This research reflects my commitment to innovation and my belief in the transformative power of AI in education.
Beyond academics, I actively engage in the tech community by managing a newsletter that highlights the latest trends in technology and AI. I am working on several impactful initiatives, including Ask-Tutor, a personalized learning system that combines specialized LLMs, RAG, and AI agents to enhance education in mathematics and computer science. I am also exploring renewable energy forecasting for my doctoral research, which aligns with my broader vision of using AI to promote sustainability and innovation.
I strive to bridge the gap between technology and its practical applications, particularly in fields like education, renewable energy, and beyond. My goal is to contribute to a future where technology not only solves problems but also empowers individuals and communities to thrive.
"""
}

### Execution dans la partie suivante

2025-01-14 16:22:28,865 - 26696 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
